# Pandas and dataframes

# Pandas and dataframes

* Numpy is limited by the basic limit of having to have the same type for every element of an ndarray. 
* a typical spreadsheet has one type per column. 
* Thus, Numpy does not correctly express those. 
* The `DataFrame` concept completely embodies the concept of a spreadsheet with multiple column types. 

These workbooks are based upon [pandas in 10 minutes](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html) and subsequent detailed tutorials.

# Pandas and numpy
* Pandas is -- in fact -- based upon `numpy` and `numpy.ndarray` concepts. 
* The convenience of Pandas comes from being higher-level. 
* In a detailed analysis, `numpy.ndarray` is often still required, e.g., for TensorFlow and related tools. 
* Thus, one has to understand "both levels of abstraction."

# Pandas vs. Numpy

| numpy | pandas |
|-------|--------|
| Strength is multidimensional arrays and tensors | Strength is spreadsheets and time series |
| Assumes integer axes | Allows axes based upon timestamp and other indexes |
| Assumes homogeneous types | Allows heterogeneous column types |
| Row queries are complex | Rpw queries are simple |
| Rudimentary csv handling | Advanced csv handling includes all special cases | 
 
# Why most people use Pandas
* Most data is in spreadsheets. 
* Unlike numpy, pandas makes reading spreadsheets trivial. 

# My advice 
* if your data is csv, then *read it into pandas and then reformat it into the appropriate numpy objects as needed.* 
Consider: 

In [ ]:
%pycat data1.csv

In [ ]:
import pandas as pd
data1 = pd.read_csv('data1.csv')
data1

# Observations
* Pandas is tuned to read MS Excel spreadsheets. 
* You can thus stop worrying about commas in strings. 
* Printouts are made pretty via correct pretty-printing calls. 
* Row number/index is listed to left. 

# What happened to `numpy`? 
* `data1` is a `DataFrame`. 
* Each of the columns are represented in `numpy`

Consider

In [ ]:
data1['heading1']

In [ ]:
data1.heading1

In [ ]:
type(data1)

In [ ]:
data1['heading1'].values

*Numpy is still underneath!*

# The reality of higher-level abstractions
* Do one thing well. 
* Are relatively poor at doing other things. 

# Pandas is
* excellent at parsing spreadsheets. 
* relatively poor at numerical operations, at which `numpy` excels. 
* Absolutely horrible at tensors. My advice is simply *don't!* Use `numpy` instead. 

# Pandas joys
Compared to `numpy`
* Columns are accessed by name. 
* Everything prints prettily. 
* Row selection is intuitive.

Consider: 


In [ ]:
data1[data1.heading1 > 1 ]

In [ ]:
data1['sum'] = data1.heading1 + data1.heading2
data1

In [ ]:
data1['approved'] = True
data1

# Observations
* Can access columns as if they are dictionaries. 
* Can access columns as if they are class members. 
* Can do column arithmetic. Result is a new column. 
* Can set a new column to the same value for all rows. 
* Can add columns to the `DataFrame` dynamically by using a new keyword for each new column. 

In [ ]:
data1.loc[data1.heading1 > 2, 'heading2']

In [ ]:
data1.loc[data1.heading1 > 2, 'heading2'] = 200
data1[data1.heading1 > 2]

# Whoa there! 

* What just happened? 
* You might recall that one of the major pains in `numpy` is that row data is immutable. 
* Here we managed to set a row and column based upon conditions upon all rows and columns. 
* `data1.loc[<row selector>, <column selector>] = <value>`
* This is difficult to understand, but really powerful. 
* It's also not all-powerful, and what it can't do is important. 

# A tale of Lvalues and Rvalues 
* At its core, Pandas very heavily uses the hacks available in Python classes. 
* These allow it to control which expressions are Lvalues and which are Rvalues. 
* An *Lvalue* is anything that can be on the left of the = sign in an assignment. 
* An *Rvalue* can be on the right hand side of the = sign in an assignment. 
* For the most part, *Lvalues are Rvalues*, but not vice-versa. 
* But, in Pandas, there is an active tension between 
    
    * Selecting data and 
    * Setting data. 

* That plays out by defining *different dyntaxes for setting and selection.*

# What this means in practice is that:
* One should be wary of placing expressions like df[..] on the left-hand side of the =, 
* because *there is a distinct syntax df.loc[...] that is designed for that!*

# An aside: how we control Lvalues and Rvalues in Python
* Python classes allow one to define methods that are different for whether the object is on the left-hand or right-hand side of the = sign. 
* Here is a simple demo

In [ ]:
class Foo(): 
    items = []
    def __getitem__(self, index): 
        print("I'm getting the the value at index {}".format(index))
        return self.items[index]
    def __setitem__(self, index, value): 
        print("I'm setting item at index {} to {}".format(index, value))
        while (len(self.items) < index+1): 
            self.items.append(None)
        self.items[index] = value

f = Foo()
f[4] = 'yo'  # f,__setitem__(4, 'yo')
print(f.items)
f[4]  # f.__getitem__(4)

# This insane little class
* implements a *self-extending list*. 
* values that are not defined are set to `None`.
* Without this intervention, if `f` were a regular list, this code would result in a runtime error. 

# The Lvalue/Rvalue minefield
* When learning Pandas and specifically `DataFrame`s, it's really difficult to keep straight what can be on the left-hand-side of the = sign in an assignment. 
* This can be a coding minefield, where assignment statements can "blow up" when you least expect them to do so. 
* Some things that look like Lvalues actually are.
    * Assigning a list of values to a whole column. 
* Some things that look like Lvalues are not, e.g., 
    * Assigning a value to part of a column. 

# Let's put this into practice.  

First, let's load some interesting data into a DataFrame: 

In [ ]:
towns = pd.read_csv('2010_Population_By_Town.csv')
print("First 10 rows are:")
towns.loc[0:9]  # First 10 rows. Remove qualifier to see all 

(source: US census, state of Conn, data.gov) 

1. Write an expression for rows for towns with population above 100000

In [ ]:
# your answer
towns[towns['2010 Population']>100000]

2. Write an expression for all towns whose name starts with 'C'. Hint: use >= 'C', < 'D' to select. 

In [ ]:
# your answer: 
towns[(towns['TOWN'] >= 'C') & (towns['TOWN'] < 'D')]


3. Write code to create a new column `Cool` and mark `Clinton` and `Wolcott` as `Cool` by setting their `Cool` columns to `True` and everyone else's to `False`.


In [ ]:
# Your answer: 
towns['Cool'] = False
towns.loc[towns.TOWN == 'Clinton', 'Cool'] = True
towns.loc[towns.TOWN == 'Wolcott', 'Cool'] = True

In [ ]:
# use this to test your answer
towns[towns.Cool == True]

4. List all towns with at least 14 characters in their names. Hint: you can apply `.str.len()` to a column to get its length as a string.

In [ ]:
towns[towns['TOWN'].str.len() >= 14]

Consider the following additional table: 

In [ ]:
tax = pd.read_csv('2012_Retail_Sales_By_Town_ALL_NAICS.csv', engine='python', skipfooter=8)
print("First 10 rows of tax are:")
tax.loc[0:9, :]  # first 10 rows; you can remove the qualifier to see more of the table. 

5. Compute "Tax per capita" as a new column by dividing "Total Tax Due" by "Number of Taxpayers". There is whitespace at the end of the label for "Total Tax Due"! 

In [ ]:
# Note the following before beginning: 
list(tax)

In [ ]:
# Your solution: 
tax['Tax per capita'] = tax["Total Tax Due **(Excluding Tax at 9.35% Rate) "]/tax["Number of Taxpayers"]
tax.loc[0:9]  # first 10 columns of solution

In [ ]:
list(tax)

6. List the towns in which the tax per capita is larger than $80,000 per entity (Gasp! This is sales tax! Entities can be businesses, though).

In [ ]:
# Your solution. 
tax[tax["Tax per capita"] > 60000]

# When you're done, submit the notebook

1. **Run all the cells in order.**

2. Submit the notebook by saving it as PDF. 
    * In the cluster environment, it's File | Print (Save as PDF) and submit to [Gradescope](https://www.gradescope.com/courses/182658)<sup>&dagger;</sup>, 
    * On other versions, it may be File | Download As (PDF) and then submit to [Gradescope](https://www.gradescope.com/courses/182658)<sup>&dagger;</sup>.

<sup>&dagger;</sup>To submit to Gradescope, log into the website, add course 9W7PW3 (if not already added) and submit. The assignment name should match the name of this notebook.